In [ ]:
!apt-get update
!apt-get install libturbojpeg
!pip install -qU git+git://github.com/lilohuang/PyTurboJPEG.git
    
!pip install -q guppy3

### Just run through all the data using open. Looks like a memory leak.

Note this is not a tqdm issue. You can check by removing it.

You can also try any of these with `cv2` instead of `TurboJPEG` and the effect should be the same.

In [ ]:
import gc

import pandas as pd
from tqdm.auto import tqdm
from turbojpeg import TurboJPEG
import cv2

JPEG = TurboJPEG()

train_df = pd.read_csv('../input/pe-train-csv/train.csv')

for path in tqdm(train_df.image_path):
    # choose whether to try with cv2 or TurboJPEG()
#     img = cv2.imread(path)
    with open(path, 'rb') as f:
        img = JPEG.decode(f.read())

I'm no expert at memory profiling but what's even more confusing is that I can't see where the memory went. The `Total size` is still around 1.5 GB.

In [ ]:
h = hpy()
print(h.heap())

### BUT, try the same again but this time loading the same file each loop. No leak.

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
from turbojpeg import TurboJPEG
import cv2

JPEG = TurboJPEG()

train_df = pd.read_csv('../input/pe-train-csv/train.csv')

for path in tqdm(train_df.image_path):
    # choose whether to try with cv2 or TurboJPEG()
#     img = cv2.imread(path)
    with open('../input/pe-train-512x512-fold-1-batch-5/000233653a7b.jpg', 'rb') as f:
        img = JPEG.decode(f.read())

### Going back to loading different files... Okay so what if I try some variations of `del` and `gc.collect()`

Putting in a `del img` doesn't help. And I can't tell if I add a `gc.collect()` on top of that because the loop takes 20x longer. So I settle for `gc.collect()` every 1000 iterations. Seems the problem is still there.

In [ ]:
import gc

import pandas as pd
from tqdm.auto import tqdm
from turbojpeg import TurboJPEG
import cv2

JPEG = TurboJPEG()

train_df = pd.read_csv('../input/pe-train-csv/train.csv')

for i, path in enumerate(tqdm(train_df.image_path)):
    # choose whether to try with cv2 or TurboJPEG()
#     img = cv2.imread(path)
    with open(path, 'rb') as f:
        img = JPEG.decode(f.read())
    del img
    if i%1000 == 0:
        gc.collect()

### Time to get creative? Nope, there's still a memory leak

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from turbojpeg import TurboJPEG
import cv2

JPEG = TurboJPEG()

train_df = pd.read_csv('../input/pe-train-csv/train.csv')

img = np.zeros(shape=(512, 512, 3)).astype(np.uint8)

for path in tqdm(train_df.image_path):
    img *= 0
    # choose whether to try with cv2 or TurboJPEG()
#     img += cv2.imread(path)
    with open(path, 'rb') as f:
        img += JPEG.decode(f.read())